In [1]:
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm

In [2]:
# from transformers import pipeline

2023-03-23 10:28:46.963099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [389]:
# Load three datasets: dataset contains all the data without labels, dataset_train_eval contains data with labels
# test contains data without labels
from datasets import load_dataset
from datasets import DatasetDict
dataset = load_dataset("csv",  data_files="../indexing/output.csv")
# We split the data test:eval 70:30 
dataset_train_eval = (load_dataset("csv", data_files="./balanced_labels_posneg.csv", split = "train").train_test_split(test_size=0.3))
dataset_test_eval = dataset_train_eval["test"].train_test_split(test_size=0.5)
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_eval['train'],
    'test': dataset_test_eval['test'],
    'valid': dataset_test_eval['train']})
test = load_dataset("csv", data_files="./test.csv")
# dataset["train"][100]

Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-df29f072a3b1628c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-19f9a1a68d359820/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-fe9974d2b3317d75/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [333]:
# Rename the column label to label_fine_grained [0,5]
# dataset_train_eval = dataset_train_eval.rename_column("label", "label_fine_grained")

In [390]:
# Rename the column label_posneg to label [0, 2]
# dataset_train_eval = dataset_train_eval.rename_column("sentiment", "label")
train_test_valid_dataset = train_test_valid_dataset.rename_column("sentiment", "label")

In [392]:
# Check the dataset_train_eval
train_test_valid_dataset["train"]

Dataset({
    features: ['Unnamed: 0', 'TEXT', 'label'],
    num_rows: 700
})

In [393]:
test

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'DATE', 'AUTHOR', 'TEXT', 'FAVORITE', 'year', 'manufacturer', 'model', 'Initial Index'],
        num_rows: 39347
    })
})

In [394]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [397]:
# Tokenizer: Hello, How are You: [Hello, How, Are, You]
# Specify Max Length 42 for now, the default model does not have a predefined maximum length
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_function(examples):
    return tokenizer(examples["TEXT"], padding="max_length", truncation=True, max_length=142)


tokenized_datasets_train = train_test_valid_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_labelled_test = train_test_valid_dataset['test'].map(tokenize_function, batched=True)
tokenized_datasets_val = train_test_valid_dataset['test'].map(tokenize_function, batched=True)
tokenized_dataset_test = test['train'].map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-19f9a1a68d359820/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-42779f2d22971cd8.arrow
Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-19f9a1a68d359820/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-689a60e44c8d417b.arrow
Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-19f9a1a68d359820/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-689a60e44c8d417b.arrow


Map:   0%|          | 0/39347 [00:00<?, ? examples/s]

In [398]:
# We load the model, and specify the num_labels as 2 [pos, neg]
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2,ignore_mismatched_sizes=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [401]:
# Evaluation metrics, we can include more such as: F-measure
import numpy as np
import evaluate

In [402]:
# import pytorch and check for GPU availability, for using torch.argmax in the future
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cpu


In [422]:
# Convert predictions to logits and use argmax function, need to change np.argmax to torch.argmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall}

In [423]:
# Hyperparameter training, for now using default hyperparameters

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", logging_steps = 100, evaluation_strategy="epoch")

In [424]:
# We can use optuna to do hyperparameter search
# def optuna_hp_space(trial):
#    return {
#        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
#        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
#    }

In [425]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    compute_metrics=compute_metrics,
)

In [426]:
trainer.train()

/Users/parkhiagarwal/opt/anaconda3/envs/info/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,No log,0.555624,0.926667,0.865854,1.000000
2,No log,0.194754,0.973333,0.946667,1.000000
3,No log,0.269488,0.960000,0.922078,1.000000


TrainOutput(global_step=264, training_loss=0.06546008948123816, metrics={'train_runtime': 1832.3764, 'train_samples_per_second': 1.146, 'train_steps_per_second': 0.144, 'total_flos': 153241634196000.0, 'train_loss': 0.06546008948123816, 'epoch': 3.0})

In [427]:
predictions = trainer.predict(tokenized_datasets_labelled_test)


In [428]:
trainer.evaluate()

{'eval_loss': 0.26948776841163635,
 'eval_accuracy': 0.96,
 'eval_precision': 0.922077922077922,
 'eval_recall': 1.0,
 'eval_runtime': 2051.8652,
 'eval_samples_per_second': 0.073,
 'eval_steps_per_second': 0.009,
 'epoch': 3.0}

In [429]:
print(predictions.predictions.shape)

(150, 2)


In [430]:
preds = np.argmax(predictions.predictions, axis=-1)

In [431]:
for x in range(10):
    print("Label: ", predictions.label_ids[x], "Prediction: ", preds[x])

Label:  0 Prediction:  0
Label:  0 Prediction:  1
Label:  1 Prediction:  1
Label:  1 Prediction:  1
Label:  0 Prediction:  0
Label:  1 Prediction:  1
Label:  0 Prediction:  0
Label:  0 Prediction:  0
Label:  1 Prediction:  1
Label:  1 Prediction:  1


In [432]:
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.96, 'f1': 0.9594594594594594}